<html>
<div>
  <img src="https://www.engineersgarage.com/wp-content/uploads/2021/11/TCH36-01-scaled.jpg" width=360px width=auto style="vertical-align: middle;">
  <span style="font-family: Georgia; font-size:30px; color: white;"> <br/> University of Tehran <br/> AI_CA0 <br/> Spring 02 </span>
</div>
<span style="font-family: Georgia; font-size:15pt; color: white; vertical-align: middle;"> low_mist - std id: 810100186 </span>
</html>

in this notebook we are to solve a searching problem with different uninformed and informed search approaches such as BFS, IDS, A* and Weighted A*

Ok first we define the directory in which we have the tests in order to run different algorithms

In [19]:
TEST_DIRECTORY = "Tests/"

## Graph and States
first we have to define the graph components.  
`Node` contains the types.  
`Edge` contains the types plus it's unsafe time list if it's rickety (it's a list that shows if we cross for the ith time we have to wait for some seconds so it's safe again to cross).  
and a dictionary mapping rickety edges to how many times they have been crossed.

In [24]:
from dataclasses import dataclass, field, fields, _MISSING_TYPE
from typing import Any, Callable
from enum import Flag, auto

class EdgeType(Flag):
    NORMAL = auto()
    RICKETY = auto()

@dataclass
class Edge:
    destination: int
    type: EdgeType
    wait_for: int
    weight: int
    id: int
    
class NodeType(Flag):
    NORMAL = auto()
    PIZZA = auto()
    STUDENT = auto()

@dataclass
class Node:
    edges: list[Edge]
    type: NodeType

now that we have edges and nodes we define the graph that contains some extra information compared to normal graphs namely, order of students, map from pizza to students.

In [21]:
class Graph:
    '''this class is only the graph connection of problem not agent's state'''
    def __init__(self, n : int):
        self.num_of_nodes = n
        self.nodes = [Node([], NodeType.NORMAL) for _ in range(n)]
        # self.rackety_edges: set(int) = set()
        # self.edges: list[tuple(int, int)] = list()
        # self.pizzas_location: set[int] = set()
        self.students_location: set[int] = set()
        self.pizzas_corresponding_students: dict[int, int] = dict()
        self.priority_queues = list[tuple[int, int]] = list()
        
    # def add_edges(self, origin: int, destination: int, weight: int = 0): 
    #     self.edges.append((origin, destination))
    #     self.nodes[origin].edges.append(Edge(destination, EdgeType.NORMAL, 0, weight)) #zero is for waiting time
    #     self.nodes[destination].edges.append(Edge(origin, EdgeType.NORMAL, 0, weight))
        
    # def add_rickety_edges(self, origin: int, destination: int, waiting_time: int, weight: int = 0):  
    #     self.edges.append((origin, destination))
    #     self.nodes[origin].edges.append(Edge(destination, EdgeType.RICKETY, waiting_time, weight)) #zero is for waiting time
    #     self.nodes[destination].edges.append(Edge(origin, EdgeType.RICKETY, waiting_time, weight))
        
    def add_edges(self, origin: int, destination: int, type: EdgeType, id: int, wait_for: int = 0, weight: int = 0): 
        # self.rackety_edges.append((origin, destination))
        self.nodes[origin].edges.append(Edge(destination, type, wait_for, weight, id)) 
        self.nodes[destination].edges.append(Edge(origin, type, wait_for, weight, id))
        
    def add_pizza_and_student(self, student_position: int, pizza_position: int):
        self.students_location.add(student_position)
        self.pizzas_corresponding_students[pizza_position] = student_position
        
    def add_priority(self, priority: tuple[int, int]):
        self.priority_queues.append(priority)

now it's time to describe the states in which our agents will be during execution of search.  
this `State` class contains information about the world such as our position and left pizzas' locations and so forth, about agent itself like the path it has taken to get here and such things.  

the properties in our states are two types:
* criteria in determining uniqueness
    * `agent_position`: which shows the agent's position.
    * `left_pizzas`: which depicts the left pizzas position and their corresponding student positions.
    * `time_elapsed`: the amount of time (in seconds) that has passed since we started the search.
    <!-- * `pizzas_carrying`: a dictionary containing the pizzas that we are carrying right now with their destination (note: it's said in problem that we can only carry one pizza at a time but we imagine that we can carry all of them and if we decide to deliver some pizza we will drop all other pizzas). -->
    * `pizza_carrying`: the tuple which contains the pizza position and the student to whom we have to deliver the pizza. (if it exists)
* not important in determining uniqueness
    * `path`: which demonstrates the path we have taken to get here
    * `satisfied_students`: which indicates the students are given their pizzas
    * `wait_till_for_rickety_edges`: a map for keep tracking of time that we have to wait
    <!-- * `pizzas_passed`: number of pizzas that we have passed -->

In [25]:
@dataclass 
class State:
    agent_position: int
    left_pizzas: dict[int, int]
    time_elapsed: int = field(default_factory = 0)
    # pizzas_carrying: dict[int, int] = dict()
    pizza_carrying = tuple[int, int] = field(default_factory = lambda: tuple(None, None))
    path: list[int] = field(default_factory = lambda: list())
    satisfied_students: set[int] = field(default_factory = lambda: set())
    pizzas_passed: int = field(default_factory = 0)
    available_time_edge: dict[int, int] = field(default_factory = lambda: dict())
    
    # def __post_init__(self):
    #     for field in fields(self):
    #         if not isinstance(field.default, _MISSING_TYPE) and getattr(self, field.name) is None:
    #             setattr(self, field.name, field.default)

    def _tuple(self):
        # return (self.agent_position, self.time_elapsed, self.pizzas_carrying, self.left_pizzas)
        return (self.agent_position, self.time_elapsed, self.left_pizzas)
    
    def __eq__(self, other: Any) -> bool:
        return (isinstance(other, State)) and (hash(self._tuple) == hash(other._tuple))

next we define some useful functions for our agent.  
- `has_reached_goad`: simply checks whether there is any pizza left or not.
- `initial_state`: to find out where we are at start of the game
- `actions`: this function gets an state and returns the list of all actions that is available

In [ ]:
from copy import deepcopy
class Agent:
    @staticmethod
    def has_reached_goal(state: State) -> bool:
        return len(state.left_pizzas) == 0

    @staticmethod
    def initial_state(graph: Graph, start_position: int) -> State:
        return State(
            agent_position = start_position,
            left_pizzas =  graph.pizzas_corresponding_students, 
            path = [start_position])  
              
    @staticmethod
    def actions(graph: Graph, state: State) -> list[State]:
        ans = list()
        for edge in graph.nodes[state.agent_position].edges:
            new_state = deepcopy(state)
            new_agent_position = edge.destination
            if EdgeType.RICKETY in edge.type:
                if not edge.id in state.available_time_edge or \
                    state.available_time_edge[edge.id] > state.time_elapsed:
                    state.available_time_edge[edge.id] = state.time_elapsed + edge.wait_for
                else:
                    new_agent_position = state.agent_position # cause we won't move
            
            new_state.path.append(new_agent_position)
            # TODO: add other actions based on node type             
            new_state.time_elapsed += 1
            ans.append(new_state)
              
                
            
            